<a href="https://colab.research.google.com/github/chinmayee1201/Youtube-clone/blob/main/StackOverflow1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers tensorflow

In [ ]:
import os
os.environ["TF_KERAS"] = "1"

In [ ]:
!pip install -q pandas
!pip install -q scikit-learn
!pip install -q tensorflow
!pip install -q transformers

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Questions.csv.xlsx", engine='openpyxl')
df.to_csv("Questions.csv", index=False)

In [ ]:
 df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Tags.csv.xlsx", engine='openpyxl')
df.to_csv("Tags.csv", index=False)

In [ ]:
df_posts = pd.read_csv('Questions.csv')   # assume columns like ['PostID', 'Text']

# Load tags
df_tags = pd.read_csv('Tags.csv')

<ipython-input-8-fb44b8eb20ba>:1: DtypeWarning: Columns (0,1,4,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_posts = pd.read_csv('Questions.csv')   # assume columns like ['PostID', 'Text']


In [ ]:
df = pd.merge(df_posts, df_tags, on='Id')

In [ ]:
import ast
import re

def safe_parse_tags(tag_str):
    try:
        # Skip if it's already a list
        if isinstance(tag_str, list):
            return tag_str
        # Fix missing quotes: convert [python, keras] → ['python', 'keras']
        tag_str = re.sub(r"(?<!['\"])(\b\w+\b)(?!['\"])", r"'\1'", tag_str)
        return ast.literal_eval(tag_str)
    except Exception as e:
        print(f"Skipping invalid tag: {tag_str} → Error: {e}")
        return []

In [ ]:
df['Tag'] = df['Tag'].apply(safe_parse_tags)

Streaming output truncated to the last 5000 lines.
Skipping invalid tag: 'jquery'-'autocomplete' → Error: malformed node or string on line 1: <ast.Constant object at 0x7a9d8e184a60>
Skipping invalid tag: 'apple'-'push'-'notifications' → Error: malformed node or string on line 1: <ast.BinOp object at 0x7a9d8e1864d0>
Skipping invalid tag: 'google'-'maps' → Error: malformed node or string on line 1: <ast.Constant object at 0x7a9d8e184eb0>
Skipping invalid tag: 'google'-'maps'-'api'-'3' → Error: malformed node or string on line 1: <ast.BinOp object at 0x7a9d8e186ce0>
Skipping invalid tag: 'ruby'-'on'-'rails' → Error: malformed node or string on line 1: <ast.BinOp object at 0x7a9d8e3009a0>
Skipping invalid tag: 'ruby'-'on'-'rails' → Error: malformed node or string on line 1: <ast.BinOp object at 0x7a9d8e1af520>
Skipping invalid tag: 'django'-'models' → Error: malformed node or string on line 1: <ast.Constant object at 0x7a9d8e1ac4c0>
Skipping invalid tag: 'django'-'templates' → Error: malfo

In [ ]:
print(df.columns)

Index(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Tag'],
      dtype='object')


In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['Body'])
sequences = tokenizer.texts_to_sequences(df['Body'])
X = pad_sequences(sequences, maxlen=200)

# Binarize multi-label targets
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Tag'])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=128, input_length=200),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(mlb.classes_), activation='sigmoid')  # multi-label
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X, y, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
16633/16633 ━━━━━━━━━━━━━━━━━━━━ 5223s 314ms/step - accuracy: 0.0921 - loss: 0.2778 - val_accuracy: 0.1134 - val_loss: 0.2619
Epoch 2/5
16633/16633 ━━━━━━━━━━━━━━━━━━━━ 5238s 313ms/step - accuracy: 0.1316 - loss: 0.2550 - val_accuracy: 0.1320 - val_loss: 0.2562
Epoch 3/5
16633/16633 ━━━━━━━━━━━━━━━━━━━━ 5082s 304ms/step - accuracy: 0.1309 - loss: 0.2488 - val_accuracy: 0.1209 - val_loss: 0.2546
Epoch 4/5
16633/16633 ━━━━━━━━━━━━━━━━━━━━ 5114s 307ms/step - accuracy: 0.1266 - loss: 0.2450 - val_accuracy: 0.1435 - val_loss: 0.2539
Epoch 5/5
15112/16633 ━━━━━━━━━━━━━━━━━━━━ 7:12 284ms/step - accuracy: 0.1271 - loss: 0.2428